In [ ]:
!pip install spacy-sentence-bert
!pip install scikit-learn

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

datasets_donwload_urllink = "https://drive.google.com/file/d/10dNvuX8Tk5xgoiCN3ohpyKzaFbYz-r8N/view?usp=drive_link"

In [5]:
file_id = '10dNvuX8Tk5xgoiCN3ohpyKzaFbYz-r8N'
url = f'https://drive.google.com/uc?id={file_id}'
df = pd.read_csv(url)

In [8]:
df_initial = df[['interactionid', 'User']]
df_initial.head(2)

,interactionid,User
0,5c69b568-4dd6-41c6-903a-90ab47256acf,"Hi Sarah, my name is Alice. What are the inter..."
1,a79cec64-9bd7-49a0-9657-81c80fd5ea4c,"Hi Sarah, my name is David, How long does it t..."


In [36]:
df['label'].value_counts()

,count
label,
AccountDetails,414
Offers,285
Loan,227
HomeLoan,224
AutoLoan,156
TechSupport,130
Sales,81
Grievances,33
DebitCard,22


In [11]:
print(df_initial.shape)

(1588, 2)


Model part

In [17]:
import spacy_sentence_bert
from sklearn.model_selection import train_test_split

In [18]:
nlp = spacy_sentence_bert.load_model('en_stsb_distilbert_base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/489 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
df['vector'] = df['User'].apply(lambda x: nlp(x).vector)

In [21]:
#train
X_train, X_test, y_train, y_test = train_test_split(df['vector'].tolist(), df['label'].tolist(), test_size=0.33, random_state=42)

In [24]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [25]:
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f"model trained with svc accuracy: {accuracy_score(y_test, y_pred)}")

0.8171428571428572


In [26]:
clf1 = RandomForestClassifier(max_depth=9, random_state=0)
clf1.fit(X_train, y_train)
y_pred1 = clf1.predict(X_test)
print(f"model trained with random forest accuracy: {accuracy_score(y_test, y_pred1)}")

model trained with svc accuracy: 0.8


Prediction evaluation

In [33]:
#print(df.loc[0:2, "User"])
test_interactions = [
  "Hi Sarah, my name is Michael. I tried to log into my online banking, but it keeps saying my credentials are incorrect. Can you help me regain access?",
  "Hi Sarah, my name is Jasmine. My debit card was declined at the store, even though I have enough balance. Can you check what might be wrong?",
  "Hi Sarah, my name is Tom. I noticed two identical transactions on my statement from yesterday. Could you help me verify if I was charged twice?",
  "Hi Sarah, my name is Priya. I received a notification about a suspicious login attempt on my account. Can you help me understand what steps I should take?",
  "Hi Sarah, my name is Omar. I'm thinking of opening a savings account for my child. Could you provide some information on the options available?"
]

topics = ['Account', 'Access', 'Debit/Credit Card Issues', 'Transactions & Payments', 'Account Inquiries', 'Loan and Credit Services', 'Fraud and Security Concerns', 'New Account', 'Product Inquiries',
          'Fees and Charges', 'General Financial Advice', 'Appointments', 'Escalations']


In [34]:
for headline, topic in zip(test_interactions, topics):
  print(headline)
  print(f"True Label: {topic}, Predicted Label: {clf.predict(nlp(headline).vector.reshape(1, -1))[0]} \n")

Hi Sarah, my name is Michael. I tried to log into my online banking, but it keeps saying my credentials are incorrect. Can you help me regain access?
True Label: Account, Predicted Label: TechSupport 

Hi Sarah, my name is Jasmine. My debit card was declined at the store, even though I have enough balance. Can you check what might be wrong?
True Label: Access, Predicted Label: TechSupport 

Hi Sarah, my name is Tom. I noticed two identical transactions on my statement from yesterday. Could you help me verify if I was charged twice?
True Label: Debit/Credit Card Issues, Predicted Label: Offers 

Hi Sarah, my name is Priya. I received a notification about a suspicious login attempt on my account. Can you help me understand what steps I should take?
True Label: Transactions & Payments, Predicted Label: TechSupport 

Hi Sarah, my name is Omar. I'm thinking of opening a savings account for my child. Could you provide some information on the options available?
True Label: Account Inquiries, 